In [87]:
%matplotlib widget
%load_ext autoreload
%autoreload 2
from IPython.display import clear_output
import os
import sys

# sys.path.insert(0, os.getenv('HOME')+'/pycode/MscThesis/')
# sys.path.insert(0,r'C:\Users\coren\Documents\PhD\Code\AMFtrack')

import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_data_info,
    update_plate_info,
    get_current_folders,
    get_folders_by_plate_id,
)


import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.image_processing.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det

# from amftrack.pipeline.functions.image_processing.experiment_class_surf import Experiment
from amftrack.pipeline.launching.run import (
    run_transfer,
)
from amftrack.pipeline.launching.run_super import run_parallel_transfer

import dropbox
from amftrack.util.dbx import upload_folders, download, read_saved_dropbox_state, save_dropbox_state, load_dbx, download, get_dropbox_folders, get_dropbox_video_folders
from subprocess import call
import logging

logging.basicConfig(stream=sys.stdout, level=logging.debug)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**To transfer data from dropbox to surfsara**
- select the folders of interest within the folders of dropbox

In [88]:
directory_targ = "/gpfs/scratch1/shared/amftrackflow/Drp_downs/20230307_Plate033/"

if not os.path.exists(directory_targ):
    os.makedirs(directory_targ)

all_folders_drop, excel_drop = get_dropbox_video_folders("/DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/", True)
# folders_drop = all_folders_drop.loc[all_folders_drop["Plate number"] == "452"]
# all_folders_drop = pd.read_json("/gpfs/home6/svstaalduine/dropbox_file_structure/data_nile_red.json")

# for xl_adress in excel_drop:
#     file_name = xl_adress.split('/')[-1]
    
#     download(xl_adress, directory_targ+file_name)
clear_output(wait=False)

all_folders_drop

,folder,Plate number,Date Imaged,tot_path_drop,video
0,20230307_Plate033_02/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,02/Img/
1,20230307_Plate033_01/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,01/Img/
2,20230307_Plate033_03/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,03/Img/
3,20230307_Plate033_04/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,04/Img/
4,20230307_Plate033_05/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,05/Img/
5,20230307_Plate033_06/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,06/Img/
6,20230307_Plate033_07/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,07/Img/
7,20230307_Plate033_08/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,08/Img/
8,20230307_Plate033_09/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,09/Img/
9,20230307_Plate033_10/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,10/Img/


In [89]:
# directory_targ = "/gpfs/scratch1/shared/amftrackflow/Rachael_dataset/"
# all_folders_drop.to_json("/gpfs/home6/svstaalduine/dropbox_file_structure/data_nile_red.json")

# save_dropbox_state("/DATA/FLUORESCENCE/DATA_NileRed/20230201_Plate552/", is_video=True)

folders_drop = all_folders_drop.loc[all_folders_drop["Plate number"].ge("031")]
# folders_drop = folders_drop.loc[folders_drop["Date Imaged"] == "20221208"]

In [90]:
folders_drop = folders_drop.sort_values(by=['folder'])

In [91]:
# FINAL CHECK

print(directory_targ)
folders_drop

/gpfs/scratch1/shared/amftrackflow/Drp_downs/20230307_Plate033/


,folder,Plate number,Date Imaged,tot_path_drop,video
1,20230307_Plate033_01/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,01/Img/
0,20230307_Plate033_02/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,02/Img/
2,20230307_Plate033_03/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,03/Img/
3,20230307_Plate033_04/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,04/Img/
4,20230307_Plate033_05/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,05/Img/
5,20230307_Plate033_06/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,06/Img/
6,20230307_Plate033_07/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,07/Img/
7,20230307_Plate033_08/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,08/Img/
8,20230307_Plate033_09/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,09/Img/
9,20230307_Plate033_10/Img/,033,20230307,DATA/MYRISTATE/DATA/2_weeks/20230307_Plate033/...,10/Img/


In [92]:
run_parallel_transfer(
    "from_drop.py",
    [directory_targ],
    folders_drop,
    1,
    "10:00:00",
    "transfer_test"
)


Submitted batch job 2791700


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.
sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2791701


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2791703


...

sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2791725


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


Submitted batch job 2791726
Submitted batch job 2791727


sbatch: Single-node jobs run on a shared node by default. Add --exclusive if you want to use a node exclusively.
sbatch: A full node consists of 32 CPU cores, 229376 MiB of memory and 0 GPUs and can be shared by up to 32 jobs.
sbatch: By default shared jobs get 7168 MiB of memory per CPU core, unless explicitly overridden with --mem-per-cpu, --mem-per-gpu or --mem.
sbatch: You will be charged for 0.03125 node, based on the number of CPUs, GPUs and the amount memory that you've requested.


In [93]:
for xl_adress in excel_drop:
    file_name = xl_adress.split('/')[-1]
    
    download(xl_adress, directory_targ+file_name)

2023-05-24 12:03:44,265-[INFO]- dropbox:390 -> Refreshing access token.
2023-05-24 12:03:44,412-[INFO]- dropbox:474 -> Request to files/download
